In [4]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [5]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]
characters_list = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters_list))
index_to_char = dict((i, c) for i, c in enumerate(characters_list))
seQ_LENGTH = 40
STEP_SIZE = 3
sentences = []
next_characters = []
for i in range(0, len(text) - 40, 3):
    sentences.append(text[i: i + 40])
    next_characters.append(text[i + 40])
x = np.zeros((len(sentences), seQ_LENGTH, len(characters_list)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters_list)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1


In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(seQ_LENGTH, len(characters_list))))
model.add(Dense(len(characters_list)))
model.add(Activation('softmax'))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer = RMSprop(learning_rate=0.01))


In [18]:
model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 102s 154ms/step - loss: 2.5130
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 97s 149ms/step - loss: 1.7630
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 100s 154ms/step - loss: 1.6015
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 148s 163ms/step - loss: 1.5255


In [20]:
model.save('textgenerator.keras')

In [27]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [36]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - seQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + seQ_LENGTH]
  generated = sentence
  for i in range(length):
    x = np.zeros((1, seQ_LENGTH, len(characters_list)))
    for t, character in enumerate(sentence):
        x[0, t, char_to_index[character]] = 1

    predictions = model.predict(x, verbose=0)[0]
    next_index = sample(predictions, 0.1)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character


  return generated

print('--------0.2--------')
print(generate_text(300, 0.2))
print('--------0.4--------')
print(generate_text(300, 0.4))
print('--------0.6--------')
print(generate_text(300, 0.6))
print('--------0.8--------')
print(generate_text(300, 0.8))
print('--------1.0--------')
print(generate_text(300, 1.0))



--------0.2--------
 good counsel may the cause remove.

benvolio:
who is the brother of the brother of the seace,
and thou shall be the words the breath, thoughts the words,
and thou see the sear the sear the from the seace
that shall the words the words and make thee in the brother,
and i wast the words the brother of the crown,
and thou not the brother of
--------0.4--------


florizel:
old sir, i know
she prizes not the words the brother of the crown,
and i wast thou see the words the words to the seack
the sear the from the seach of the brother,
and i wast thou see the sear the seace and thoughts,
and i wast thou see the words the seace of the brother,
and i wast thou wearing heart with his forth,
and i was
--------0.6--------
ear me with patience but to speak a words,
and i wast thou see the words of the words,
and thou shall stay the sear the from the seace,
and i wast thou see the words the words the words,
and i wast thou wearing hand to the from the seace,
and thou sees the s